In [8]:
import cv2
from pathlib import Path
import numpy as np
import pandas as pd
import os
from glob import glob
import natsort
import matplotlib.pyplot as plt
from scipy.stats import mode, skew, kurtosis, entropy

from skimage.feature import graycomatrix, graycoprops
from skimage import feature


In [30]:
def calculate_hist_features(hist):

    feature_vector = []
    feature_vector.append(np.mean(hist)) # Mean
    # feature_vector.append(mode(hist)[0][0]) # Mode
    feature_vector.append(np.std(hist)) # Standard deviation
    feature_vector.append(skew(hist)[0]) # Skewness
    feature_vector.append(np.sum(hist**2)) # Energy
    feature_vector.append(entropy(hist, base=2)[0]) # Entropy
    feature_vector.append(kurtosis(hist)[0]) # Kurtosis

    print(sum(feature_vector), feature_vector)
    
    return np.array(feature_vector)

In [36]:


    # Calculate normalized histograms for each channel
    hist_r = (cv2.calcHist([r], [0], None, [n_bins], [0, 256])) / N
    hist_g = (cv2.calcHist([g], [0], None, [n_bins], [0, 256])def extract_color_features(img):

    b, g, r = cv2.split(img) # Split the image into the B, G, R channels
    n_bins = 256 # Number of bins
    height, width, _ = img.shape
    N = height * width # Number of pixels) / N
    hist_b = (cv2.calcHist([b], [0], None, [n_bins], [0, 256])) / N

    color_features = np.concatenate((calculate_hist_features(hist_r), calculate_hist_features(hist_g), calculate_hist_features(hist_b)))

    return color_features

In [32]:
ROOT_PATH = Path(Path(os.getcwd())/"../challenge1")
TRAIN_PATH = ROOT_PATH/"train"
VAL_PATH = ROOT_PATH/"val"

train_nevus = sorted(glob(str(TRAIN_PATH/'nevus/*')))
train_others = sorted(glob(str(TRAIN_PATH/'others/*')))

In [37]:
feature_names = [
    'Mean_R', 'Std_Dev_R', 'Skewness_R', 'Energy_R', 'Entropy_R', 'Kurtosis_R', # 'Mode_R',
    'Mean_G', 'Std_Dev_G', 'Skewness_G', 'Energy_G', 'Entropy_G', 'Kurtosis_G', # 'Mode_G',
    'Mean_B', 'Std_Dev_B', 'Skewness_B', 'Energy_B', 'Entropy_B', 'Kurtosis_B'  # 'Mode_B'
]
# Create an empty list to store DataFrames
df_list = []

for index, name_image in enumerate(train_nevus): 
    img = cv2.imread(train_nevus[index])

    color_features = extract_color_features(img)
    color_features_df = pd.DataFrame([color_features], columns=feature_names)
    # Append the extracted features DataFrame to the list
    df_list.append(color_features_df)
    # Concatenate all DataFrames in the list into a single DataFrame
    combined_df = pd.concat(df_list, ignore_index=True)

    if index==10:
        break

    


22.227718255795004 [0.00390625, 0.010585630636430511, 3.6276733337766007, 0.03259247744855968, 5.590849142092733, 12.962111421840683]
17.42797736917685 [0.0039062499999999996, 0.009305990238258238, 3.0542571885467154, 0.026076222304526747, 5.809914276820267, 8.524517441267083]
16.558731953387372 [0.00390625, 0.008910922955256842, 2.9264476272374655, 0.024233814266117974, 5.937940151651526, 7.657293187277007]
119.85140518821048 [0.00390625, 0.017169906205454805, 9.834630630315408, 0.07937650385065353, 5.2616380254642845, 104.65468387237468]
120.77432562483537 [0.00390625, 0.017139064789346188, 9.888050161620134, 0.07910562071447202, 5.355121489385132, 105.43100303832628]
123.84543787360212 [0.00390625, 0.01703421291480546, 10.069122724574354, 0.07818833886449283, 5.52037335509575, 108.15681299215272]
30.209711809996975 [0.00390625, 0.012454944886026277, 4.481782906063582, 0.043618416941171745, 5.263000257724443, 20.40494903438175]
23.242620835943058 [0.00390625, 0.011277852604945751, 3.

In [6]:
combined_df

# example for 10 images

,Mean_R,Std_Dev_R,Skewness_R,Energy_R,Entropy_R,Kurtosis_R,Mean_G,Std_Dev_G,Skewness_G,Energy_G,Entropy_G,Kurtosis_G,Mean_B,Std_Dev_B,Skewness_B,Energy_B,Entropy_B,Kurtosis_B
0,0.003922,0.010604,3.619345,0.032592,5.590849,12.895754,0.003922,0.009321,3.046806,0.026076,5.809914,8.475214,0.003922,0.008925,2.919211,0.024234,5.937940,7.611478
1,0.003922,0.023569,14.520333,0.145576,4.899479,219.925234,0.003922,0.023547,14.562177,0.145305,4.992963,220.780089,0.003922,0.023471,14.704948,0.144398,5.158188,223.748367
2,0.003922,0.012477,4.472047,0.043618,5.262965,20.309510,0.003922,0.011297,3.822801,0.036467,5.465146,13.823373,0.003922,0.010740,3.678852,0.033334,5.557813,12.902844
3,0.003922,0.009742,3.646792,0.028123,5.786762,13.100083,0.003922,0.007161,2.956125,0.016998,6.516997,8.400862,0.003922,0.006655,2.726460,0.015214,6.681399,6.528725
4,0.003922,0.012490,4.104263,0.043701,5.079543,16.740156,0.003922,0.009265,3.038064,0.025811,5.826128,8.379944,0.003922,0.008238,2.791942,0.021227,6.093673,6.961385
5,0.003922,0.015129,4.650682,0.062289,4.460245,21.557520,0.003922,0.009187,2.587978,0.025445,5.649410,5.528495,0.003922,0.008566,2.519454,0.022633,5.877623,5.224228
6,0.003922,0.006453,2.034746,0.014541,6.485447,3.165081,0.003922,0.004931,1.174141,0.010122,6.847938,0.242262,0.003922,0.004648,1.111005,0.009432,6.948098,0.204191
7,0.003922,0.011072,3.719229,0.035184,5.521027,12.992840,0.003922,0.009640,3.384455,0.027617,5.834003,11.096366,0.003922,0.008038,2.610306,0.020397,6.161692,5.732068
8,0.003922,0.013019,4.305456,0.047144,4.970579,18.639251,0.003922,0.008609,2.479370,0.022822,5.820418,5.013118,0.003922,0.007054,1.789862,0.016611,6.163183,1.812330
9,0.003922,0.007891,3.105439,0.019801,6.339891,9.737459,0.003922,0.006014,2.205198,0.013143,6.766562,4.283821,0.003922,0.005225,1.581977,0.010883,6.918801,1.525514


In [24]:
combined_df[0]

KeyError: 0

#### GLCM Texture feature implementation 

In [21]:
def extract_glcm_features(image):    
    if not isinstance(image, np.ndarray):
        raise ValueError("Input must be a numpy array")

    if image.dtype != np.uint8:
        # If the image is not 8-bit, convert it to 8-bit
        # important as we set the levels for glcm to 256 for 8-bit images
        image = image.astype(np.uint8)

    # Convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Compute GLCM matrix
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(gray_image, distances, angles, levels=256, symmetric=True, normed=True)
    
    # Extract texture features from GLCM matrix
    features = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    texture_features = np.concatenate([graycoprops(glcm, feature).ravel() for feature in features])
    
    return texture_features


In [22]:
texture_features = []

for index, name_image in enumerate(train_nevus): 
    img = cv2.imread(train_nevus[index])

    texture_features.append(extract_glcm_features(img))

    if index==10:
        break

len(texture_features[0])

24

In [23]:
texture_features[0]

array([4.22450751e+00, 5.61807913e+00, 3.63345212e+00, 6.53887139e+00,
       1.38064181e+00, 1.55583731e+00, 1.27876021e+00, 1.72673089e+00,
       5.06314360e-01, 4.76821501e-01, 5.27080984e-01, 4.45922596e-01,
       7.23539365e-02, 6.94901683e-02, 7.48672134e-02, 6.56716429e-02,
       9.98628570e-01, 9.98179604e-01, 9.98820776e-01, 9.97881244e-01,
       5.23509212e-03, 4.82888349e-03, 5.60509964e-03, 4.31276468e-03])